In [1]:
#!pip install

In [2]:
import networkx as nx
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl

from tqdm import tqdm

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix, average_precision_score

from gensim.models.word2vec import Word2Vec

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score


In [3]:
#Reading dataset

df = pd.read_csv("dataset/CollegeMsg.txt", sep=" ",names=['src','dst','time'])

In [4]:
#Dividing the Dataset in Snapshots

df1=df.iloc[0:11968]
df2=df.iloc[11968:23935]
df3=df.iloc[23935:35902]
df4=df.iloc[35902:47869]
df5=df.iloc[47869:]
df2 = df2.reset_index(drop=True)
df3 = df3.reset_index(drop=True)
df4 = df4.reset_index(drop=True)
df5 = df5.reset_index(drop=True)

#print(df1,df2,df3,df4,df5)


In [6]:
#G
G= nx.Graph()

src= df['src']
dst= df['dst']

for i in range (src.size):
        s=src[i]
        d=dst[i]
        G.add_edges_from([(s,d)])

G_nodes= list(G.nodes())

for i in range(src.size):
    if src[i] in G_nodes:
        continue
    else:
        G.add_node(src[i])

for i in range(dst.size):
    if dst[i] in G_nodes:
        continue
    else:
        G.add_node(dst[i])
        
G_nodes= list(G.nodes())

#nx.draw_networkx(G)

In [7]:
#G1
G1= nx.Graph()

src1= df1['src']
dst1= df1['dst']

for i in range (src1.size):
        s=src1[i]
        d=dst1[i]
        G1.add_edges_from([(s,d)])

G1_nodes= list(G1.nodes())

for i in range(src1.size):
    if src1[i] in G1_nodes:
        continue
    else:
        G1.add_node(src1[i])

for i in range(dst1.size):
    if dst1[i] in G1_nodes:
        continue
    else:
        G1.add_node(dst1[i])
        
G1_nodes= list(G1.nodes())


#nx.draw_networkx(G1)

In [8]:
#G2
G2= nx.Graph()

src2= df2['src']
dst2= df2['dst']

for i in range (src2.size):
        s=src2[i]
        d=dst2[i]
        G2.add_edges_from([(s,d)])

G2_nodes= list(G2.nodes())

for i in range(src2.size):
    if src2[i] in G2_nodes:
        continue
    else:
        G2.add_node(src2[i])

for i in range(dst2.size):
    if dst2[i] in G2_nodes:
        continue
    else:
        G2.add_node(dst2[i])
        
G2_nodes= list(G2.nodes())

#nx.draw_networkx(G2)

In [9]:
#G3
G3= nx.Graph()

src3= df3['src']
dst3= df3['dst']

for i in range (src3.size):
        s=src3[i]
        d=dst3[i]
        G3.add_edges_from([(s,d)])

G3_nodes= list(G3.nodes())

for i in range(src3.size):
    if src3[i] in G3_nodes:
        continue
    else:
        G3.add_node(src3[i])

for i in range(dst3.size):
    if dst3[i] in G3_nodes:
        continue
    else:
        G3.add_node(dst3[i])
        
G3_nodes= list(G3.nodes())

#nx.draw_networkx(G3)

In [10]:
#G4
G4= nx.Graph()

src4= df4['src']
dst4= df4['dst']

for i in range (src4.size):
        s=src4[i]
        d=dst4[i]
        G4.add_edges_from([(s,d)])

G4_nodes= list(G4.nodes())

for i in range(src4.size):
    if src4[i] in G4_nodes:
        continue
    else:
        G4.add_node(src4[i])

for i in range(dst4.size):
    if dst4[i] in G4_nodes:
        continue
    else:
        G4.add_node(dst4[i])
        
G4_nodes= list(G4.nodes())

#nx.draw_networkx(G4)

In [11]:
#G5
G5= nx.Graph()

src5= df5['src']
dst5= df5['dst']

for i in range (src5.size):
        s=src5[i]
        d=dst5[i]
        G5.add_edges_from([(s,d)])

G5_nodes= list(G5.nodes())

for i in range(src5.size):
    if src5[i] in G5_nodes:
        continue
    else:
        G5.add_node(src5[i])

for i in range(dst5.size):
    if dst5[i] in G5_nodes:
        continue
    else:
        G5.add_node(dst5[i])
        
G5_nodes= list(G5.nodes())


#nx.draw_networkx(G5)

In [12]:
#Adding nodes from master graph to G5
for i in range(len(G_nodes)):
    if G_nodes[i] in G5_nodes:
        continue
    else:
        G5.add_node(G_nodes[i])
        
G5_nodes= list(G5.nodes())

In [12]:
# Preparing Train and Test Datasets
from gae.preprocessing import mask_test_edges
np.random.seed(0) # make sure train-test split is consistent between notebooks

adj_sparse = nx.to_scipy_sparse_matrix(G5)

adj_train, train_edges, train_edges_false, val_edges, val_edges_false, \
    test_edges, test_edges_false = mask_test_edges(adj_sparse, test_frac=.3, val_frac=.0, prevent_disconnect = True)

print ("Total nodes:", adj_sparse.shape[0])
print ("Total edges:", int(adj_sparse.nnz/2)) # adj is symmetric, so nnz (num non-zero) = 2*num_edges
print ("Training edges (positive):", len(train_edges))
print ("Training edges (negative):", len(train_edges_false))
print ("Test edges (positive):", len(test_edges))
print ("Test edges (negative):", len(test_edges_false))


Total nodes: 1026
Total edges: 2817
Training edges (positive): 1972
Training edges (negative): 1972
Test edges (positive): 845
Test edges (negative): 845


In [13]:
#Adding all nodes to snapshot

#G1
for i in range(len(G_nodes)):
    if G_nodes[i] in G1_nodes:
        continue
    else:
        G1.add_node(G_nodes[i])
        
G1_nodes= list(G1.nodes())

#G2
for i in range(len(G_nodes)):
    if G_nodes[i] in G2_nodes:
        continue
    else:
        G2.add_node(G_nodes[i])
        
G2_nodes= list(G2.nodes())

#G3
for i in range(len(G_nodes)):
    if G_nodes[i] in G3_nodes:
        continue
    else:
        G3.add_node(G_nodes[i])
        
G3_nodes= list(G3.nodes())

#G4
for i in range(len(G_nodes)):
    if G_nodes[i] in G4_nodes:
        continue
    else:
        G4.add_node(G_nodes[i])
        
G4_nodes= list(G4.nodes())

In [15]:
# #Removing test edges from Snapshots

#G1

G1_edges=G1.edges()

for i in range(len(test_edges)):
    if test_edges[i] in G1_edges:
        G1.remove_edge(*test_edges[i])


 #G2
    
G2_edges=G2.edges()


for i in range(len(test_edges)):
    if test_edges[i] in G2_edges:
        G2.remove_edge(*test_edges[i])


#G3

G3_edges=G3.edges()



for i in range(len(test_edges)):
    if test_edges[i] in G3_edges:
        G3.remove_edge(*test_edges[i])

#G4

G4_edges=G4.edges()

for i in range(len(test_edges)):
    if test_edges[i] in G4_edges:
        G4.remove_edge(*test_edges[i])
        

In [14]:
#Getting node embeddings

from node2vec import Node2Vec

#G1
node2vec_1 = Node2Vec(G1, dimensions=128, walk_length=80, num_walks=12)
n2v_model_1 = node2vec_1.fit(window=10, min_count=1)

#G2
node2vec_2 = Node2Vec(G2, dimensions=128, walk_length=80, num_walks=12)
n2v_model_2 = node2vec_2.fit(window=10, min_count=1)

#G3
node2vec_3 = Node2Vec(G3, dimensions=128, walk_length=80, num_walks=12)
n2v_model_3 = node2vec_3.fit(window=10, min_count=1)

#G4
node2vec_4 = Node2Vec(G4, dimensions=128, walk_length=80, num_walks=12)
n2v_model_4 = node2vec_4.fit(window=10, min_count=1)

Computing transition probabilities:   0%|          | 0/1899 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|███████████████████████████████████████████████████████| 12/12 [00:04<00:00,  2.46it/s]


Computing transition probabilities:   0%|          | 0/1899 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|███████████████████████████████████████████████████████| 12/12 [00:05<00:00,  2.36it/s]


Computing transition probabilities:   0%|          | 0/1899 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|███████████████████████████████████████████████████████| 12/12 [00:05<00:00,  2.29it/s]


Computing transition probabilities:   0%|          | 0/1899 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|███████████████████████████████████████████████████████| 12/12 [00:05<00:00,  2.12it/s]


In [15]:
#Getting edge embeddings

def get_edge_embeddings(edge_list,n2v_model):
    embs = []
    for edge in edge_list:
        node1 = edge[0]
        node2 = edge[1]
        emb1 = n2v_model.wv[node1]
        emb2 = n2v_model.wv[node2]
        edge_emb = np.add(emb1, emb2)
        embs.append(edge_emb)
    embs = np.array(embs)
    return embs


In [18]:
#Train dataset

#Positive

pos_train_embs_1= get_edge_embeddings(train_edges,n2v_model_1)
pos_train_embs_2= get_edge_embeddings(train_edges,n2v_model_2)
pos_train_embs_3= get_edge_embeddings(train_edges,n2v_model_3)
pos_train_embs_4= get_edge_embeddings(train_edges,n2v_model_4)

#Negative

neg_train_embs_1= get_edge_embeddings(train_edges_false,n2v_model_1)
neg_train_embs_2= get_edge_embeddings(train_edges_false,n2v_model_2)
neg_train_embs_3= get_edge_embeddings(train_edges_false,n2v_model_3)
neg_train_embs_4= get_edge_embeddings(train_edges_false,n2v_model_4)

In [19]:
#Test dataset

#Positive

pos_test_embs_1= get_edge_embeddings(test_edges,n2v_model_1)
pos_test_embs_2= get_edge_embeddings(test_edges,n2v_model_2)
pos_test_embs_3= get_edge_embeddings(test_edges,n2v_model_3)
pos_test_embs_4= get_edge_embeddings(test_edges,n2v_model_4)

#Negative

neg_test_embs_1= get_edge_embeddings(test_edges_false,n2v_model_1)
neg_test_embs_2= get_edge_embeddings(test_edges_false,n2v_model_2)
neg_test_embs_3= get_edge_embeddings(test_edges_false,n2v_model_3)
neg_test_embs_4= get_edge_embeddings(test_edges_false,n2v_model_4)

In [20]:
#Train Dataset
train_edges_emb_1=np.hstack([pos_train_embs_1,pos_train_embs_2,pos_train_embs_3,pos_train_embs_4])
train_edges_label_1=np.ones(len(train_edges))
train_edges_emb_2=np.hstack([neg_train_embs_1,neg_train_embs_2,neg_train_embs_3,neg_train_embs_4])
train_edges_label_2=np.zeros(len(train_edges_false))

train_edges_label= np.concatenate([train_edges_label_1,train_edges_label_2])
train_edges_emb= np.concatenate([train_edges_emb_1,train_edges_emb_2])

In [21]:
#Test Dataset
test_edges_emb_1=np.hstack([pos_test_embs_1,pos_test_embs_2,pos_test_embs_3,pos_test_embs_4])
test_edges_label_1=np.ones(len(test_edges))
test_edges_emb_2=np.hstack([neg_test_embs_1,neg_test_embs_2,neg_test_embs_3,neg_test_embs_4])
test_edges_label_2=np.zeros(len(test_edges_false))

test_edges_label= np.concatenate([test_edges_label_1,test_edges_label_2])
test_edges_emb= np.concatenate([test_edges_emb_1,test_edges_emb_2])

In [22]:
# Train logistic regression classifier on train-set edge embeddings
from sklearn.linear_model import LogisticRegression
edge_classifier = LogisticRegression(max_iter=3000,random_state=0)
edge_classifier.fit(train_edges_emb, train_edges_label)

# Predicted edge scores: probability of being of class "1" (real edge)
test_preds = edge_classifier.predict_proba(test_edges_emb)[:, 1]
test_roc = roc_auc_score(test_edges_label, test_preds)
test_ap = average_precision_score(test_edges_label, test_preds)
print(test_roc)
print(test_ap)

0.9424347886978748
0.9189273499849676


In [23]:
edge_classifier_1= RandomForestClassifier(n_estimators = 50)
edge_classifier_1.fit(train_edges_emb, train_edges_label)

test_preds_1 = edge_classifier_1.predict_proba(test_edges_emb)[:, 1]
test_roc_1 = roc_auc_score(test_edges_label, test_preds_1)
test_ap_1 = average_precision_score(test_edges_label, test_preds_1)

print(test_roc_1)
print(test_ap_1)

0.9397226987850565
0.9206430494644313


In [24]:
from sklearn.ensemble import GradientBoostingClassifier
edge_classifier_2 = GradientBoostingClassifier(n_estimators=50, learning_rate=0.5, max_depth=8, random_state=0).fit(train_edges_emb, train_edges_label)
edge_classifier_2 = GradientBoostingClassifier(n_estimators=50, learning_rate=0.5, max_depth=8, random_state=0).fit(train_edges_emb, train_edges_label)

test_preds_2 = edge_classifier_2.predict_proba(test_edges_emb)[:, 1]
test_roc_2 = roc_auc_score(test_edges_label, test_preds_2)
test_ap_2 = average_precision_score(test_edges_label, test_preds_2)

print(test_roc_2)
print(test_ap_2)

0.9343846503973952
0.9140562452920873


In [25]:
# #Train XGBoost classifier on train-set edge embeddings

# import xgboost as xgb

# xgb_cl = xgb.XGBClassifier()

# from sklearn.metrics import accuracy_score

# xgb_cl = xgb.XGBClassifier()
# xgb_cl.fit(train_edges_emb,train_edges_label)
# preds = xgb_cl.predict(test_edges_emb)

# print(accuracy_score(test_edges_label, preds))

In [26]:
# #Train Random Forest classifier on train-set edge embeddings

# #classifier
# clf1 = RandomForestClassifier()
 
# # parameters
# param = {'n_estimators' : [10,50,100], 'max_depth' : [5,10,15]}
 
# # model
# grid_clf_acc1 = GridSearchCV(clf1, param_grid = param)
 
# # train the model
# grid_clf_acc1.fit(train_edges_emb,train_edges_label)
 
# print('Grid best parameter (max. accuracy): ', grid_clf_acc1.best_params_)
# print('Grid best score (accuracy): ', grid_clf_acc1.best_score_)
 
# # alternative metric to optimize over grid parameters: AUC
# grid_clf_auc1 = GridSearchCV(clf1, param_grid = param, scoring = 'roc_auc')
# grid_clf_auc1.fit(train_edges_emb,train_edges_label)
# predict_proba = grid_clf_auc1.predict_proba(test_edges_emb)[:,1]
 
# print('Test set AUC: ', roc_auc_score(test_edges_label, predict_proba))
# print('Grid best parameter (max. AUC): ', grid_clf_auc1.best_params_)
# print('Grid best score (AUC): ', grid_clf_auc1.best_score_)

In [27]:
# #Train Gradient Boost classifier on train-set edge embeddings

# # classifier
# clf2 = GradientBoostingClassifier()
 
# # parameters
# param = {'learning_rate' : [.05,.1]}
 
# # model
# grid_clf_acc2 = GridSearchCV(clf2, param_grid = param)
 
# # train the model
# grid_clf_acc2.fit(train_edges_emb,train_edges_label)
 
# print('Grid best parameter (max. accuracy): ', grid_clf_acc2.best_params_)
# print('Grid best score (accuracy): ', grid_clf_acc2.best_score_)
 
# # alternative metric to optimize over grid parameters: AUC
# grid_clf_auc2 = GridSearchCV(clf2, param_grid = param, scoring = 'roc_auc')
# grid_clf_auc2.fit(train_edges_emb,train_edges_label)
# predict_proba = grid_clf_auc2.predict_proba(test_edges_emb)[:,1]
 
# print('Test set AUC: ', roc_auc_score(test_edges_label, predict_proba))
# print('Grid best parameter (max. AUC): ', grid_clf_auc2.best_params_)
# print('Grid best score (AUC): ', grid_clf_auc2.best_score_)
